# 카카오 로컬 API
- 도로명 주소 --> 위도, 경도

In [1]:
import pandas as pd

In [2]:
with open('kakaoapikey.txt') as f:
    api_key = f.read()

In [3]:
len(api_key)

32

In [4]:
import requests
from urllib.parse import quote

In [5]:
addr = '서울특별시 중구 세종대로 110(태평로1가)'
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A4%91%EA%B5%AC%20%EC%84%B8%EC%A2%85%EB%8C%80%EB%A1%9C%20110%28%ED%83%9C%ED%8F%89%EB%A1%9C1%EA%B0%80%29'

In [6]:
# Authorization: KakaoAK {REST_API_KEY}
result = requests.get(url,
    headers={"Authorization": f'KakaoAK {api_key}'}).json()
result

{'documents': [{'address': {'address_name': '서울 중구 태평로1가 31',
    'b_code': '1114010300',
    'h_code': '1114055000',
    'main_address_no': '31',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '명동',
    'region_3depth_name': '태평로1가',
    'sub_address_no': '',
    'x': '126.977829174031',
    'y': '37.5663174209601'},
   'address_name': '서울 중구 세종대로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 중구 세종대로 110',
    'building_name': '서울특별시청',
    'main_building_no': '110',
    'region_1depth_name': '서울',
    'region_2depth_name': '중구',
    'region_3depth_name': '태평로1가',
    'road_name': '세종대로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.977829174031',
    'y': '37.5663174209601',
    'zone_no': '04524'},
   'x': '126.977829174031',
   'y': '37.5663174209601'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [7]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
lat, lng

(37.5663174209601, 126.977829174031)

## 공공기관.csv파일에 위도, 경도 추가하기

In [15]:
df = pd.read_csv('C:/Workspace/02.DataAnalysis/04.공공API_및_지도시각화/data/공공기관.csv')

In [22]:
lat_list=[]
lng_list=[]

for i in df.index:
    url = f'{search_url}?query={quote(df.도로명주소[i])}'
    result = requests.get(url,
                headers={"Authorization": f'KakaoAK {api_key}'}).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

df['위도'] = lat_list
df['경도'] = lng_list

In [23]:
df

,공공기관,도로명주소,위도,경도
0,서울특별시청,서울특별시 중구 세종대로 110(태평로1가),37.566317,126.977829
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654519,127.056297
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543


In [24]:
df.to_csv('공공기관2.csv',index=False)

### 공공기관 표시하기

In [17]:
import folium

In [26]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=12)
for i in df.index:
    folium.Circle(radius=200,
    location=[df.위도[i],df.경도[i]],
    popup=folium.Popup(f'{df.도로명주소[i]}',max_width=200),
    tooltip=df.공공기관[i],
    color='#3186cc',
    fill=True
    ).add_to(map)
title='<h3 align="center" style="font-size:20px">서울시 공공기관 안내</h3>'
map.get_root().html.add_child(folium.Element(title))
map